In [1]:
import torch
import numpy as np

In [18]:
x = torch.ones(2,2, requires_grad=True)
y = x + 2
y.requires_grad = True
z = y * x
f = torch.relu(z)

b = torch.tensor([[1., 2.],[3,4]], dtype=torch.float)
print(b)

f.backward(b)

RuntimeError: you can only change requires_grad flags of leaf variables.